In [3]:
import os
os.environ['ESMFMKFILE'] = '/home/malasawedah/.conda/envs/test2/lib/esmf.mk'

# Now you can import the ESMF library
#import ESMF


In [22]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
#import xvec
import shapely
from shapely import Point, box
import esmpy
import xagg as xa

In [18]:
# Load some climate data as an xarray dataset
ds = xr.open_dataset('./data/climate_data/tas_Amon_CCSM4_rcp85_monthavg_20700101-20991231.nc')

# Load US counties shapefile as a geopandas GeoDataFrame
gdf = gpd.read_file('./data/geo_data/UScounties.shp')

In [23]:
ds

<xarray.Dataset>
Dimensions:   (lon: 288, lat: 192, month: 12, bnds: 2)
Coordinates:
    height    float64 ...
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... -5.0 -3.75 -2.5 -1.25
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * month     (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: bnds
Data variables:
    lat_bnds  (month, lat, bnds) float64 ...
    lon_bnds  (month, lon, bnds) float64 -0.625 0.625 0.625 ... -1.875 -0.625
    tas       (month, lat, lon) float32 ...

In [19]:
# Calculate overlaps
weightmap = xa.pixel_overlaps(ds,gdf)

creating polygons for each pixel...
calculating overlaps between pixels and output polygons...
success!


In [20]:
# Aggregate
aggregated = xa.aggregate(ds,weightmap)

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating tas...
all variables aggregated to polygons!


In [21]:
ds_out = aggregated.to_dataset()
ds_out

<xarray.Dataset>
Dimensions:     (poly_idx: 3141, month: 12)
Coordinates:
  * poly_idx    (poly_idx) int64 0 1 2 3 4 5 6 ... 3135 3136 3137 3138 3139 3140
  * month       (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    NAME        (poly_idx) object 'Lake of the Woods' 'Ferry' ... 'Broomfield'
    STATE_NAME  (poly_idx) object 'Minnesota' 'Washington' ... 'Colorado'
    STATE_FIPS  (poly_idx) object '27' '53' '53' '53' ... '02' '02' '02' '08'
    CNTY_FIPS   (poly_idx) object '077' '019' '065' '047' ... '240' '068' '014'
    FIPS        (poly_idx) object '27077' '53019' '53065' ... '02068' '08014'
    tas         (poly_idx, month) float64 263.9 268.8 274.0 ... 276.4 270.4